In [64]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Veri yükleme ve ön işleme

In [48]:
train_dir = r'C:\Users\hilal\OneDrive\Belgeler\GenerativeAI\Keras\Data\train'
val_dir = r'C:\Users\hilal\OneDrive\Belgeler\GenerativeAI\Keras\Data\valid'
test_dir = r'C:\Users\hilal\OneDrive\Belgeler\GenerativeAI\Keras\Data\test'

In [40]:
img_height, img_width = 224, 224
batch_size = 32

In [41]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [42]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [53]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 613 images belonging to 4 classes.


In [54]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 72 images belonging to 4 classes.


In [56]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 315 images belonging to 4 classes.


# Modeli oluşturma

In [57]:
base_model = EfficientNetV2L(weights = "imagenet", include_top = False, input_shape = (img_height, img_width,3))

473176280/473176280 [==============================] - 159s 0us/step


In [58]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
predictions = Dense(train_generator.num_classes, activation = "softmax")(x)

In [59]:
model = Model(inputs = base_model.input, outputs = predictions)

In [60]:
for layers in base_model.layers:
    layers.trainable = False

In [61]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = "categorical_crossentropy", metrics = ["accuracy"])

# Modeli eğitme

In [62]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

In [63]:
history = model.fit(train_generator,
                    steps_per_epoch = train_generator.samples//batch_size,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples//batch_size,
                   epochs = 50,
                   callbacks = [checkpoint, early_stopping])

Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 1.3790 - accuracy: 0.2995

C:\Users\hilal\anaconda3\envs\Hilal\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 157s 7s/step - loss: 1.3790 - accuracy: 0.2995 - val_loss: 1.3681 - val_accuracy: 0.3281
Epoch 2/50
19/19 [==============================] - 119s 6s/step - loss: 1.3596 - accuracy: 0.3339 - val_loss: 1.3740 - val_accuracy: 0.3281
Epoch 3/50
19/19 [==============================] - 94s 5s/step - loss: 1.3460 - accuracy: 0.3442 - val_loss: 1.3803 - val_accuracy: 0.4531
Epoch 4/50
19/19 [==============================] - 89s 5s/step - loss: 1.3380 - accuracy: 0.3569 - val_loss: 1.3311 - val_accuracy: 0.4219
Epoch 5/50
19/19 [==============================] - 84s 4s/step - loss: 1.3132 - accuracy: 0.4114 - val_loss: 1.3534 - val_accuracy: 0.4375
Epoch 6/50
19/19 [==============================] - 97s 5s/step - loss: 1.3029 - accuracy: 0.3890 - val_loss: 1.3518 - val_accuracy: 0.4375
Epoch 7/50
19/19 [==============================] - 100s 5s/step - loss: 1.2816 - accuracy: 0.4114 - val_loss: 1.2842 - val_accuracy: 0.4375
Epoch 8/50
19/19 [==========

# Modeli değerlendirme

### En iyi modeli yükleme

In [65]:
model.load_weights('best_model.h5')

### Test seti üzerinde değerlendirme

In [66]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc:.2f}')

9/9 [==============================] - 37s 4s/step - loss: 1.6298 - accuracy: 0.2465
Test accuracy: 0.25
